In [2]:
import pandas as pd
import numpy as np
import neuroCombat as nC
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from scipy.stats import ranksums, ttest_ind, ttest_rel, ks_2samp
import os
import Nestedcombat as nested
from sklearn.preprocessing import StandardScaler
from collections import Counter


#import GMMComBat as gmmc

In [14]:
#Load the data
main_dir= "/home/ulaval.ca/lesee/projects/Project-NLST/"

#features_cohort1_T0 = pd.read_csv(os.path.join(main_dir,'data/radiomicsfeatures_kheops-NLST-Dmitrii-Cohort2_Laptop_v2-cleaned-T2-ML.csv'))
#batch_cohort1_T0 = pd.read_csv(os.path.join(main_dir,'data/cohort2-T2-batch.csv'))
#clinical_cohort1_T0 = pd.read_csv(os.path.join(main_dir,'data/Cohort2_clinical.csv'))
features = pd.read_csv(os.path.join(main_dir,'data/radiomicsfeatures_kheops-NLST-Dmitrii-Cohort1_2_Laptop_v2.csv'))
batch = pd.read_csv(os.path.join(main_dir,'data/combat_params_cohort1_2.csv'))
clinical = pd.read_csv(os.path.join(main_dir,'data/Cohort1_2_clinical.csv'))
print(np.shape(features))
print(np.shape(batch))

(1694, 1239)
(1959, 14)


In [15]:
value_counts = batch['ConvolutionKernel'].value_counts()
# Identify values that occur exactly once
print(value_counts.index)
value_counts = batch['SliceThickness'].value_counts()
# Identify values that occur exactly once
print(value_counts.index)
value_counts = batch['Manufacturer'].value_counts()
# Identify values that occur exactly once
print(value_counts.index)

Index(['STANDARD', 'B50f', 'B30f', 'BONE', 'C', 'D', 'LUNG', 'FC51', 'FC10',
       'FC30', 'B', 'FC02', 'A', 'B80f', 'FC01', 'B60f', 'B50s', 'B30s',
       'FC50', 'B70f', 'FC82'],
      dtype='object')
Float64Index([2.5, 2.0, 3.2, 1.25, 1.0, 3.0, 1.3], dtype='float64')
Index(['GE MEDICAL SYSTEMS', 'SIEMENS', 'Philips', 'TOSHIBA'], dtype='object')


In [16]:
# Define the custom function to categorize the kernel types
def categorize_kernel(kernel):
#    sharp_kernels = ['BONE', 'B80f', 'B60f', 'B50f', 'B30f', 'LUNG', 'B70f', 'B50s', 'B30s']
#    soft_kernels = ['STANDARD', 'C', 'D', 'B', 'FC51', 'FC10', 'FC30', 'FC02', 'FC01', 'A', 'FC50', 'FC82']
#    sharp_kernels = ['LUNG', 'D', 'B60f', 'FC30', 'B50f','FC51', 'BONE', 'B80f','B70f', 'B50s', 'B30s']
#    soft_kernels = ['STANDARD', 'C', 'B30f', 'FC10', 'B', 'FC02', 'FC01', 'A', 'FC50', 'FC82']
    sharp_kernels = ['D', 'B70f', 'B80f', 'BONE', 'FC82', 'LUNG', 'B60f', 'FC51', 'FC30', 'FC50']
    soft_kernels = ['B30f', 'B30s', 'FC10', 'FC02', 'FC01', 'A',  'STANDARD', 'B', 'C', 'B50s', 'B50f']    
    
    if kernel in sharp_kernels:
        return 'Sharp'
    elif kernel in soft_kernels:
        return 'Soft'
    else:
        return 'Unknown'  # For any values not explicitly listed
    
# Define the function for categorization
def categorize_thickness(thickness):
    if thickness < 2.0:
        return 'Thin'
    elif thickness <= 3.0:
        return 'Medium'
    else:
        return 'Thick'

# Apply the function to create a new column
#batch['ThicknessCategory'] = batch['SliceThickness'].apply(categorize_thickness)
batch['ThicknessCategory'] = batch['SliceThickness']


# Apply the function to the original column to create a new one
batch['KernelType'] = batch['ConvolutionKernel'].apply(categorize_kernel)

In [17]:
print(batch.columns)

Index(['PatientID', 'StudyDate', 'StudyInstanceUID', 'SeriesInstanceUID',
       'SeriesDescription', 'ImageType', 'Manufacturer', 'ModelName',
       'ConvolutionKernel', 'ReconDiameter', 'SliceThickness', 'kVp', 'mAs',
       'PixelSpacing', 'ThicknessCategory', 'KernelType'],
      dtype='object')


In [18]:
#print(batch_cohort1_T0.columns)

In [19]:
print(clinical.columns)

Index(['PatientID', 'StudyDate', 'SeriesDescription', 'Age', 'Smoking',
       'De_Stag', 'Gender', 'Lesionsize', 'Race'],
      dtype='object')


In [20]:
match_column = 'PatientID'

# Define the columns to match on as a list
match_columns = ['PatientID', 'SeriesInstanceUID']

#Perform the joins
# First, merge df1 and df2
#merged_df1_df2 = pd.merge(features_cohort1_T0, batch_cohort1_T0, on=match_column, how='outer')
merged_df1_df2 = pd.merge(features, batch, on=match_columns, how='inner')
print(merged_df1_df2.columns)
print(np.shape(merged_df1_df2))

Index(['Unnamed: 0', 'PatientID', 'StudyDate_x', 'SeriesDescription_x',
       'Kernel', 'CtSlices', 'SegSlices', 'SeriesInstanceUID',
       'diagnostics_Image-original_Mean', 'diagnostics_Image-original_Minimum',
       ...
       'Manufacturer', 'ModelName', 'ConvolutionKernel', 'ReconDiameter',
       'SliceThickness', 'kVp', 'mAs', 'PixelSpacing', 'ThicknessCategory',
       'KernelType'],
      dtype='object', length=1253)
(1694, 1253)


In [21]:
# Then, merge the result with df3
#cohort1_T0_rads_batch_clinical = pd.merge(merged_df1_df2, clinical_cohort1_T0, on=match_column, how='outer')
cohort1_2_rads_batch_clinical = pd.merge(merged_df1_df2, clinical, on=match_column, how='inner')

In [22]:
print(cohort1_2_rads_batch_clinical.columns)
print(np.shape(cohort1_2_rads_batch_clinical))

Index(['Unnamed: 0', 'PatientID', 'StudyDate_x', 'SeriesDescription_x',
       'Kernel', 'CtSlices', 'SegSlices', 'SeriesInstanceUID',
       'diagnostics_Image-original_Mean', 'diagnostics_Image-original_Minimum',
       ...
       'ThicknessCategory', 'KernelType', 'StudyDate', 'SeriesDescription',
       'Age', 'Smoking', 'De_Stag', 'Gender', 'Lesionsize', 'Race'],
      dtype='object', length=1261)
(1689, 1261)


In [23]:
batch_list = ['ThicknessCategory']
#define a string-based covariate datframe                       
covars_string = pd.DataFrame()
covars_string[categorical_cols] = filtered_data_copy[categorical_cols].copy()
covars_string[batch_list] = filtered_data_copy[batch_list].copy()

counts_batch_levels = Counter(list(covars_string["ThicknessCategory"]))

for element, count in sorted(counts_batch_levels.items()):
    print(f"{element}: {count}")

1.0: 19
1.25: 89
2.0: 663
2.5: 738
3.0: 12
3.2: 164


In [12]:
#specify the batch list                                            
#batch_list = ['Manufacturer', 'ModelName', 'ConvolutionKernel', 'SliceThickness']
#batch_list = ['ThicknessCategory', 'ConvolutionKernel', 'Manufacturer']
batch_list = ['ThicknessCategory']
# specify the categorical and continous clinical covariates
categorical_cols = ['Smoking', 'Gender']

continuous_cols = ['Age']

# Explicitly create a copy of the data to work with
filtered_data_copy = cohort1_2_rads_batch_clinical.copy()

# Initialize an empty mask (series of False values)
# This will be used to mark rows to drop
rows_to_drop = pd.Series(False, index=filtered_data_copy.index)

# Loop over each categorical column
for col in batch_list:
    # Calculate the value counts for the current column
    value_counts = filtered_data_copy[col].value_counts()
    
    # Identify values that occur exactly once
    values_to_drop = value_counts[value_counts < 5].index
    
    # Update the mask to include rows where the current column's value is in values_to_drop
    rows_to_drop |= filtered_data_copy[col].isin(values_to_drop)
    print(values_to_drop)

#remove batch levels with only one sample
# Drop rows marked in the mask
filtered_data_copy = filtered_data_copy[~rows_to_drop]
print(np.shape(filtered_data_copy))


Float64Index([1.3], dtype='float64')
(1685, 1261)


In [45]:
# Filter for rows where the Manufacturer is 'Toshiba'
toshiba_data = filtered_data_copy[filtered_data_copy['Manufacturer'] == 'TOSHIBA']

# Get unique ConvolutionKernel values used by Toshiba
toshiba_kernels = toshiba_data['ConvolutionKernel'].unique()

print("ConvolutionKernel values for Toshiba:")
print(toshiba_kernels)


ConvolutionKernel values for Toshiba:
['FC51' 'FC02' 'FC30' 'FC10' 'FC50' 'FC01' 'FC82']


In [43]:
# Filter for rows where the Manufacturer is 'Toshiba'
toshiba_data = filtered_data_copy[filtered_data_copy['Manufacturer'] == 'GE MEDICAL SYSTEMS']

# Get unique ConvolutionKernel values used by Toshiba
toshiba_kernels = toshiba_data['ConvolutionKernel'].unique()

print("ConvolutionKernel values for GE MEDICAL SYSTEMS:")
print(toshiba_kernels)

ConvolutionKernel values for GE MEDICAL SYSTEMS:
['STANDARD' 'BONE' 'LUNG']


In [44]:
# Filter for rows where the Manufacturer is 'Toshiba'
toshiba_data = filtered_data_copy[filtered_data_copy['Manufacturer'] == 'Philips']

# Get unique ConvolutionKernel values used by Toshiba
toshiba_kernels = toshiba_data['ConvolutionKernel'].unique()

print("ConvolutionKernel values for Philips:")
print(toshiba_kernels)

ConvolutionKernel values for Philips:
['C' 'D' 'A' 'B']


In [47]:
#define a string-based covariate datframe                       
covars_string = pd.DataFrame()
covars_string[categorical_cols] = filtered_data_copy[categorical_cols].copy()
covars_string[batch_list] = filtered_data_copy[batch_list].copy()

counts_batch_levels = Counter(list(covars_string["ConvolutionKernel"]))

for element, count in sorted(counts_batch_levels.items()):
    print(f"{element}: {count}")

KeyError: 'ConvolutionKernel'

In [36]:
#define a string-based covariate datframe                       
covars_string = pd.DataFrame()
covars_string[categorical_cols] = filtered_data_copy[categorical_cols].copy()
covars_string[batch_list] = filtered_data_copy[batch_list].copy()

counts_batch_levels = Counter(list(covars_string["ThicknessCategory"]))

for element, count in sorted(counts_batch_levels.items()):
    print(f"{element}: {count}")

Medium: 1404
Thick: 164
Thin: 112


In [37]:
#define a string-based covariate datframe                       
covars_string = pd.DataFrame()
covars_string[categorical_cols] = filtered_data_copy[categorical_cols].copy()
covars_string[batch_list] = filtered_data_copy[batch_list].copy()

counts_batch_levels = Counter(list(covars_string["Manufacturer"]))

for element, count in sorted(counts_batch_levels.items()):
    print(f"{element}: {count}")

GE MEDICAL SYSTEMS: 827
Philips: 168
SIEMENS: 639
TOSHIBA: 46


In [17]:
median_thickness = np.median(covars_string['SliceThickness'])
print(median_thickness)
#define a string-based covariate datframe                       
covars_string = pd.DataFrame()
covars_string[categorical_cols] = filtered_data_copy[categorical_cols].copy()
covars_string[batch_list] = filtered_data_copy[batch_list].copy()

counts_batch_levels = Counter(list(covars_string['SliceThickness']))

for element, count in sorted(counts_batch_levels.items()):
    print(f"{element}: {count}")
#    print(np.median(counts_batch_levels [0]))

KeyError: 'SliceThickness'

In [48]:

#drop columns not used in this work
#filtered_data_copy.drop(columns = ['Center', 'pixel_spacing', 'StudyInstanceUID', 'Recurrence', 'OS_days', 'PFS_days', 'manufacturer'], inplace=True)
                        
#Remove rows with missing data
#filtered_data_copy.dropna(inplace=True)
#print("Shape of the dataset with radiomic features : " , np.shape(filtered_data_copy))

# Store the feature names
feature_names = filtered_data_copy.columns.tolist()
#print(feature_names)                    
patient_name = filtered_data_copy['PatientID']     
study_date = filtered_data_copy['StudyDate_x']
series_uid = filtered_data_copy['SeriesInstanceUID']


In [49]:
#define a string-based covariate datframe                       
covars_string = pd.DataFrame()
covars_string[categorical_cols] = filtered_data_copy[categorical_cols].copy()
covars_string[batch_list] = filtered_data_copy[batch_list].copy()
print("String-based covariate columns : ", covars_string.columns)
#print(list(filtered_data_copy.columns))


String-based covariate columns :  Index(['Smoking', 'Gender', 'Manufacturer'], dtype='object')


In [50]:
#load the continous clinical covariate                       
covars_quant = filtered_data_copy[continuous_cols]

#specify the features
data_df = filtered_data_copy.drop(columns=['Unnamed: 0', 'PatientID', 'StudyDate_x', 'SeriesDescription_x', 'Kernel', 
                                           'CtSlices', 'SegSlices', 'SeriesInstanceUID', 'StudyDate_y', 'StudyInstanceUID',
                                           'SeriesDescription_y', 'ImageType', 'Manufacturer', 'ModelName', 'ConvolutionKernel', 
                                           'ReconDiameter', 'SliceThickness', 'kVp', 'mAs', 'PixelSpacing', 'StudyDate', 
                                           'SeriesDescription', 'Age', 'Smoking', 'De_Stag', 'Gender', 'Lesionsize', 'Race',
                                           'ThicknessCategory', 'KernelType'
                                          ])



#print(data_df.columns)
#Remove constant radiomic features
constant_features = data_df.columns[data_df.nunique() == 1]
data_df.drop(constant_features, axis=1, inplace=True)
data_df = data_df.reset_index(drop=True)
#dat = data_df.T
dat = data_df.T.apply(pd.to_numeric)                        
#print(dat)
#label-encode the string covariates
covars_cat = pd.DataFrame()


for col in covars_string:
    stringcol = covars_string[col]
    le = LabelEncoder()
    le.fit(list(stringcol))
    covars_cat[col] = le.transform(stringcol)
#print(np.shape(data_df))
print(np.shape(dat))
#print(data_df.columns)
print(np.shape(covars_cat))
print(covars_cat.columns)
print(np.shape(covars_quant))
print(covars_quant.columns)
#print(len(covars_cat[batch_list[0]]))

(1109, 1689)
(1689, 3)
Index(['Smoking', 'Gender', 'Manufacturer'], dtype='object')
(1689, 1)
Index(['Age'], dtype='object')


In [51]:
covars_cat_reset = covars_cat.reset_index(drop=True)
covars_quant_reset = covars_quant.reset_index(drop=True)

#concatenate the label-enoded categorical (batch+clinical) and continous clinical covariates                         
covars = pd.concat([covars_cat_reset, covars_quant_reset], axis=1)
print(categorical_cols)
print(continuous_cols)
#print(covars)
# Check if there are any NaN values in covars and batch_col
covars_nan_count = np.sum(np.sum(np.isnan(data_df)))
#batch_col_nan_count = np.sum(np.isnan(batch_col))

print("Number of NaN values in covars:", covars_nan_count)
#print("Number of NaN values in batch_col:", batch_col_nan_count)

['Smoking', 'Gender']
['Age']
Number of NaN values in covars: 0


In [53]:
print(np.shape(dat))
#Nested Combat
output_data = nested.NestedComBat(dat, covars, batch_list, categorical_cols=categorical_cols,
                                   continuous_cols=continuous_cols, drop=False, write_p=False, filepath=main_dir)
output_data_reset = output_data.reset_index(drop=True)
patient_name_reset = patient_name.reset_index(drop=True)
study_date_reset = study_date.reset_index(drop=True)
series_uid_reset = series_uid.reset_index(drop=True)
write_df = pd.concat([patient_name_reset, study_date_reset,series_uid_reset, output_data_reset], axis=1)
write_df.to_csv(os.path.join(main_dir,'results/cohort1_2_rads_combat_man_notdropped.csv'), float_format='%.7f', index= False)


#Combat
#data_combat = nC.neuroCombat(dat= dat.values,
#                          covars=covars,
#                          batch_col=batch_list[0]
#                          ,categorical_cols=categorical_cols,
#                          continuous_cols=continuous_cols
#                            )["data"]
#pd.DataFrame(data_combat).to_csv(os.path.join(main_dir,'data_combat/combat_harmonized_features_slice_new.csv'), float_format='%.7f', index= False)


(1109, 1689)
ROUND 1:
Harmonizing by Manufacturer...
ComBat with Raw Data...
[neuroCombat] Creating design matrix
[neuroCombat] Standardizing data across features
[neuroCombat] Fitting L/S model and finding priors
[neuroCombat] Finding parametric adjustments
[neuroCombat] Final adjustment of data


In [16]:
                       
#nested.feature_kstest_histograms(output_data, covars, batch_list, main_dir)

#f_dict = nested.MultiComBat(output_data.T, covars, batch_list, filepath=main_dir, categorical_cols=categorical_cols,
#                          continuous_cols=continuous_cols, write_p=True, plotting=True)
#for col in batch_list:
#    write_df = pd.concat([patient_name, f_dict[col]])
#    write_df.to_csv(os.path.join(main_dir, 'data_combat/combat_'+col+'_harmonized_features.csv'))